In [0]:
from pyspark.sql import DataFrameWriter
import pandas as pd
from pyspark.sql.functions import *

In [0]:
dbutils.secrets.listScopes()

[SecretScope(name='scope31dec')]

In [0]:
dbutils.secrets.list(scope = 'scope31dec')

[SecretMetadata(key='blobkeysecret31dec')]

In [0]:
spark.conf.set("fs.azure.account.key.healthcareblob31dec.dfs.core.windows.net",
    dbutils.secrets.get(scope="scope31dec", key="blobkeysecret31dec"))

In [0]:
display(dbutils.fs.ls("abfss://rawdata@healthcareblob31dec.dfs.core.windows.net"))

path,name,size,modificationTime
abfss://rawdata@healthcareblob31dec.dfs.core.windows.net/Patient_records.csv,Patient_records.csv,5110,1704010961000
abfss://rawdata@healthcareblob31dec.dfs.core.windows.net/claims.json,claims.json,16385,1704027744000
abfss://rawdata@healthcareblob31dec.dfs.core.windows.net/disease.csv,disease.csv,1489,1704010960000
abfss://rawdata@healthcareblob31dec.dfs.core.windows.net/group.csv,group.csv,4390,1704010960000
abfss://rawdata@healthcareblob31dec.dfs.core.windows.net/hospital.csv,hospital.csv,1328,1704010960000
abfss://rawdata@healthcareblob31dec.dfs.core.windows.net/subgroup.csv,subgroup.csv,561,1704010960000
abfss://rawdata@healthcareblob31dec.dfs.core.windows.net/subscriber.csv,subscriber.csv,12061,1704010960000


In [0]:
subgroup_data = spark.read.csv("abfss://rawdata@healthcareblob31dec.dfs.core.windows.net/subgroup.csv", header = True, inferSchema= True)

In [0]:
#subgroup_data.show(5)

In [0]:
subgroup_data.printSchema()

root
 |-- subgrp_sk: string (nullable = true)
 |-- subgrp_name: string (nullable = true)
 |-- monthly_premium: integer (nullable = true)
 |-- subgrp_id: string (nullable = true)



In [0]:
#subgroup_data.isna().sum()

In [0]:
subgroup_data.select([count(when(isnan(c) | col(c).isNull(), c)). alias(c) for c in subgroup_data.columns]).show()


+---------+-----------+---------------+---------+
|subgrp_sk|subgrp_name|monthly_premium|subgrp_id|
+---------+-----------+---------------+---------+
|        0|          0|              0|        0|
+---------+-----------+---------------+---------+



In [0]:
# check for duplicates
# drop duplicates

In [0]:
subgroup_data.groupBy('subgrp_sk').count().where("count > 1").show()

+---------+-----+
|subgrp_sk|count|
+---------+-----+
+---------+-----+



In [0]:
#subgroup_data.groupBy('subgrp_sk', 'subgrp_name', 'monthly_premium', 'subgrp_id').count().where("count > 1").show()
# commenting to optimize

In [0]:
subgroup_data = subgroup_data.drop_duplicates()

In [0]:
#subgroup_data.show(5,False)

In [0]:
subgroup_data = subgroup_data.withColumn("subgrp_id", split(subgroup_data['subgrp_id'], ","))

In [0]:
subgroup_data.show(6)

+---------+-------------------+---------------+--------------------+
|subgrp_sk|        subgrp_name|monthly_premium|           subgrp_id|
+---------+-------------------+---------------+--------------------+
|     S104|            Therapy|           1500|[GRP103, GRP113, ...|
|     S105|          Allergies|           2300|[GRP153, GRP104, ...|
|     S103|         Physiology|           2000|[GRP122, GRP108, ...|
|     S102|           Accident|           1000|[GRP110, GRP150, ...|
|     S101|Deficiency Diseases|           3000|    [GRP101, GRP105]|
|     S107|             Cancer|           3200|[GRP151, GRP131, ...|
+---------+-------------------+---------------+--------------------+
only showing top 6 rows



In [0]:
#subgroup_data.show(6, False)

In [0]:
# explode - transforms each element of the list to row
subgroup_data = subgroup_data.withColumn("subgrp_id", explode(subgroup_data['subgrp_id']) )

In [0]:
#subgroup_data.show(6)

In [0]:
#loading into staging data
#staging_path = "abfss://stagingdata@healthcareblob31dec.dfs.core.windows.net"
#subgroup_data.write.format("csv").option("header","true").mode("overwrite").option("path", staging_path).save()

In [0]:
#display(dbutils.fs.ls("abfss://stagingdata@healthcareblob31dec.dfs.core.windows.net"))

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File <command-3336075213342467>, line 1
----> 1 display(dbutils.fs.ls("abfss://stagingdata@healthcareblob31dec.dfs.core.windows.net"))

File /databricks/python_shell/dbruntime/display.py:205, in Display.display(self, input, *args, **kwargs)
    203         self.add_custom_display_data("table", serialized_plan)
    204 elif isinstance(input, list):
--> 205     self.display(self.sparkSession.createDataFrame(input))
    206 elif type(input).__module__ == 'pandas.core.frame' and type(input).__name__ == 'DataFrame':
    207     self.display(self.sparkSession.createDataFrame(input))

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_

In [0]:
#subgroup_data.show(6)

In [0]:
output_container_path = "abfss://stagingdata@healthcareblob31dec.dfs.core.windows.net"
output_blob_folder = "stagingdata/"
subgroup_data.coalesce(1).write.mode("overwrite").option("header" , "true"). format("com.databricks.spark.csv").save(output_blob_folder)
files = dbutils.fs.ls(output_blob_folder)
outputfile = [x for x in files if x.name.startswith("part-")]
dbutils.fs.mv(outputfile[0].path, "%s/subgroup_data_stage.csv"% output_container_path)

True